<a href="https://colab.research.google.com/github/u6yuvi/NER-System/blob/main/notebooks/Data_Splitting_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=4ff2eb41c7805046c61a099c4f310a23d265556a7d52276eeaa151f9b749822f
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

In [5]:
from datasets import load_dataset
dataset = load_dataset("conll2002", "es")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8324 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1518 [00:00<?, ? examples/s]

Dataset conll2002 downloaded and prepared to /root/.cache/huggingface/datasets/conll2002/es/1.0.0/a3a8a8612caf57271f5b35c5ae1dd25f99ddb9efb9c1667abaa70ede33e863e5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
label2ids = {j:i for i,j in enumerate(dataset["train"].features['ner_tags'].feature.names)}
label2ids

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [7]:
id2labels = {i:j for j,i in label2ids.items()}
id2labels

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [8]:
label_list = list(label2ids.keys())

In [9]:
from collections import Counter, defaultdict

split2freqs = defaultdict(Counter) 
for data in dataset["train"]:
  for tag in data["ner_tags"]: 
      # if tag.startswith("B"):
      #   tag_type = tag.split("-")[1] 
        split2freqs["train"][tag] += 1

pd.DataFrame.from_dict(split2freqs, orient="index")

,5,0,3,1,2,7,4,6,8
train,4913,231920,7390,4321,3903,2173,4992,1891,3212


In [66]:
text_tokens  = dataset['train'].to_pandas().tokens.tolist()

In [26]:
from sklearn import feature_extraction,neighbors
from typing import Dict, Iterable, List, Optional, Tuple, Union
from scipy import stats
from wasabi import msg

In [24]:
def _get_elements_by_idx(l: Iterable, idx: Iterable) -> Iterable:
    return [l[i] for i in idx]


In [90]:
def wasserstein(
    text: List[str],
    test_size: float = 0.2,
    leaf_size: int = 3,
    use_counts: bool = False,
    min_df: Union[int, float] = 0.10,
    n_jobs: Optional[int] = -1,
) -> Tuple[List[int], List[int]]:
    """
    Perform adversarial splitting using a divergence maximization method
    involving Wasserstein distance.
    This method approximates the test split by performing nearest-neighbor
    search on a random centroid. Based on Søgaard, Ebert et al.'s work on
    'We Need to Talk About Random Splits' (EACL 2021).
    This splitter takes in a List of text tokens and returns its training
    and test partitions.
    text (List[text]): list of text tokens to split.
    test_size (float): the number of neighbors to query.
    leaf_size (int): the leaf size parameter for nearest-neighbor search.
        High values are slower, but less memory-heavy computation.
    use_counts (bool): Use count vectors instead of initialized vectors. If no
        vectors were found, the count vectors are automatically used.
    min_df (Union[int,float]): Remove terms that appear too infrequently given a threshold.
    n_jobs (Optional[int]): Number of parallel jobs to run for neighbor search
    RETURNS the training and test partitions
    """


    nn_tree = neighbors.NearestNeighbors(
        n_neighbors=int(test_size * len(text)),
        algorithm="ball_tree",
        leaf_size=leaf_size,
        metric=stats.wasserstein_distance,
        n_jobs=n_jobs,
    )
    vectorizer = feature_extraction.text.CountVectorizer(
          dtype=np.int8,tokenizer = lambda x : x,lowercase = False,min_df = min_df
      )
      # vectorizer = feature_extraction.text.CountVectorizer(
      #     dtype=np.int8, min_df=min_df
      # )
    text_counts = vectorizer.fit_transform(text)
    word_vectors = text_counts.todense()

    msg.text(f"Performing nearest neighbor search (shape={word_vectors.shape})")
    nn_tree.fit(word_vectors)

    sampled_point = np.random.randint(
        np.asarray(word_vectors).max().max() + 1,
        size=(1, np.asarray(word_vectors).shape[1]),
    )
    nearest_neighbors = nn_tree.kneighbors(sampled_point, return_distance=False)
    test_idxs = nearest_neighbors[0]  # query only a single point

    all_idxs = set(range(len(text)))
    train_idxs = all_idxs - set(test_idxs)

    docs_train = _get_elements_by_idx(text, train_idxs)
    docs_test = _get_elements_by_idx(text, test_idxs)
    msg.text(f"Sizes after split: train ({len(docs_train)}), test ({len(docs_test)})")
    return docs_train, docs_test

In [91]:
text_tokens  = dataset['train'].to_pandas().tokens.tolist()

In [92]:
train_split,test_split  = wasserstein(text_tokens)

Performing nearest neighbor search (shape=(8324, 28))


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Sizes after split: train (6660), test (1664)
